In [1]:
%matplotlib inline
import xarray as xr
import intake
import util
import warnings
from cmip6_preprocessing.preprocessing import read_data
import numpy as np
from cmip6_preprocessing.preprocessing import rename_cmip6
import gsw
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy.ma as ma
nm = 12
import xesmf as xe
import pickle

/srv/conda/envs/notebook/lib/python3.7/site-packages/intake/source/discovery.py:136: FutureWarning: The drivers ['stac-catalog', 'stac-collection', 'stac-item'] do not specify entry_points and were only discovered via a package scan. This may break in a future release of intake. The packages should be updated.
  FutureWarning)


In [2]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster()
cluster.adapt(minimum=1, maximum=10)
cluster

In [3]:
from dask.distributed import Client
client = Client(cluster) # Connect this local process to remote workers
client 

Client Scheduler: tcp://10.32.78.45:39439 Dashboard: /user/0000-0003-4189-3928/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


# Read in CMIP6 data using intake_esm

In [4]:
def get_dictionary():
    """
    Function to get the dictionary of models and ensemble members of the historical runs 
    that have all of siconc, so, and thetao
    
    Returns the dictionary, the appropriate intake-esm catalog and the list of models needed to pass
    to the next function that gets the datasets.
    """
    
    print('opening intake-esm catalog...')
    url = "https://raw.githubusercontent.com/andrewpauling/cmip6hack-so-project/master/catalogs/pangeo-cmip6.json"
    col = intake.open_esm_datastore(url)
    print('done')
    
    cat = col.search(experiment_id=['historical'], table_id=['SImon', 'Omon'],
                 grid_label='gn')
    
    uni_dict = cat.unique(['source_id', 'experiment_id', 'table_id', 'member_id'])
    
    cat = col.search(experiment_id=['historical'], table_id=['SImon', 'Omon'],
                 grid_label='gn', variable_id=['siconc', 'thetao', 'so'])
    
    print('Find the models that have all three variables...')
    models = set(uni_dict['source_id']['values']) # all the models

    for table_id in ['SImon', 'Omon']:
        if table_id == 'SImon':
            query = dict(experiment_id='historical', table_id=table_id, 
                         variable_id='siconc', grid_label='gn')  
            cat = col.search(**query)
            models = models.intersection({model for model in cat.df.source_id.unique().tolist()})
        else:
            for variable_id in ['thetao', 'so']:
                query = dict(experiment_id='historical', table_id=table_id, 
                             variable_id=variable_id, grid_label='gn')  
                cat = col.search(**query)
                models = models.intersection({model for model in cat.df.source_id.unique().tolist()})
                
    models = list(models)
    print('Done')
    
    cat = col.search(experiment_id='historical', table_id=['Omon', 'SImon'], 
                 variable_id=['siconc', 'thetao', 'so'], grid_label='gn', source_id=models)
    
    print('Make sure all three variables have the same ensemble member...')
    filt_dict = dict()

    for model in models:
        tmp2 = cat.search(source_id=model)
        tmp2.df.head()
        members = tmp2.df['member_id']
        memlist = list()
        for member in list(members):
            a = tmp2.search(member_id=member, variable_id='siconc').df['activity_id'].empty
            b = tmp2.search(member_id=member, variable_id='thetao').df['activity_id'].empty
            c = tmp2.search(member_id=member, variable_id='so').df['activity_id'].empty
            if not a and not b and not c and member not in memlist:
                memlist.append(member)
        filt_dict[model] =  memlist
        
    print('Done')
    
    return filt_dict, cat, models
def get_datasets(filt_dict, cat, models):
    """
    Function to load the dataset dictionaries for each of the variables siconc, so, thetao. Takes in the output
    of get_dictionary()
    Returns the dataset dictonary for each variable. Separate one for each variable due to problems with intake-esm 
    for some models
    """
        
    icedict = dict()
    sodict = dict()
    thetaodict = dict()
    for model in models:
        print(model)
        tmpice = cat.search(source_id=model, member_id=filt_dict[model], variable_id='siconc')
        tmpdict_ice = tmpice.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': True},
                                             cdf_kwargs={'chunks': {}, 'decode_times': True})
        icedict.update(tmpdict_ice)
        tmpice = None
        tmpdictice = None
    
        tmpso = cat.search(source_id=model, member_id=filt_dict[model], variable_id='so')
        tmpdict_so = tmpso.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': True},
                                           cdf_kwargs={'chunks': {}, 'decode_times': True})
        sodict.update(tmpdict_so)
        tmpso = None
        tmpdictso = None
    
        tmpthetao = cat.search(source_id=model, member_id=filt_dict[model], variable_id='thetao')
        tmpdict_thetao = tmpthetao.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': True},
                                                   cdf_kwargs={'chunks': {}, 'decode_times': True})
        thetaodict.update(tmpdict_thetao)
        tmpthetao = None
        tmpdictthetao = None
    
    return icedict, sodict, thetaodict

In [5]:
[filt_dict, cat, models] = get_dictionary()

opening intake-esm catalog...
done
Find the models that have all three variables...
Done
Make sure all three variables have the same ensemble member...
Done


In [ ]:
# seemed tp have problems with this
pickle.dump( [filt_dict, cat, models] , open( "save.p", "wb" ) ) 

In [7]:
#[filt_dict, cat, models] = pickle.load( open( "save.p", "rb" ) )

In [8]:
icedict, sodict, thetaodict = get_datasets(filt_dict, cat, models)

GFDL-CM4
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
SAM0-UNICON
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
--> The keys in the returned dictionary of datasets are con

In [9]:
%%capture
# one model has inconsistencies b/w time grid so remove it
sodict.pop('CMIP.CAMS.CAMS-CSM1-0.historical.Omon.gn')
thetaodict.pop('CMIP.CAMS.CAMS-CSM1-0.historical.Omon.gn')
icedict.pop('CMIP.CAMS.CAMS-CSM1-0.historical.SImon.gn')

In [10]:
%%capture
# MIROC is weird - some NaNs at funny places in the ocean that
# mean we can't calculate the bottom properties, so exclude
sodict.pop('CMIP.MIROC.MIROC-ES2L.historical.Omon.gn')
thetaodict.pop('CMIP.MIROC.MIROC-ES2L.historical.Omon.gn')
icedict.pop('CMIP.MIROC.MIROC-ES2L.historical.SImon.gn')
sodict.pop('CMIP.MIROC.MIROC6.historical.Omon.gn')
thetaodict.pop('CMIP.MIROC.MIROC6.historical.Omon.gn')
icedict.pop('CMIP.MIROC.MIROC6.historical.SImon.gn')

In [11]:
# unify variable names and select first ensemble member
for i in thetaodict:
    thetaodict[i] = rename_cmip6(thetaodict[i]).isel(member_id = 0)
    sodict[i] = rename_cmip6(sodict[i]).isel(member_id = 0)
for i in icedict:
    icedict[i] = rename_cmip6(icedict[i]).isel(member_id = 0)

In [12]:
calc_props = {}
for i in thetaodict:    
    
    # calculate cthetao and add to ds
    cthetao = xr.apply_ufunc(gsw.CT_from_pt, sodict[i].so, thetaodict[i].thetao, dask='parallelized',
                                         output_dtypes=[float,]).rename('cthetao').to_dataset() 
    
    # calculate sigma0 and sigma2
    sigma0 = xr.apply_ufunc(gsw.density.sigma0,sodict[i].so, cthetao.cthetao, dask='parallelized', 
                        output_dtypes=[float, ]).rename('sigma0').to_dataset()                                                                          
   
    sigma2=xr.apply_ufunc(gsw.density.sigma2,sodict[i].so, cthetao.cthetao, dask='parallelized', 
                        output_dtypes=[float, ]).rename('sigma2').to_dataset()
    
    # interpolate density data
    surf_dens = sigma0.sigma0.interp(lev=10)
    
     # Calculate mixed layer depth based on density difference from 10m
    dens_diff = sigma0.sigma0 - surf_dens
    dens_diff = dens_diff.where(dens_diff > 0.03)
    mld = dens_diff.lev.where(dens_diff==dens_diff.min(['lev'])).max(['lev']).rename('mld')
    
     # get variables at bottom
    test = sigma0.sigma0 + sigma0.lev
    bottom_depth = sigma2.lev.where(test == test.max(dim='lev')).max(dim='lev').rename('bottom_depth')
    bottom_sigma2 = sigma2.sigma2.where(test == test.max(['lev'])).max(dim='lev').rename('bottom_sigma2')
    bottom_temp = thetaodict[i].thetao.where(test == test.max(['lev'])).max(dim='lev').rename('bottom_temp')
    bottom_salt = sodict[i].so.where(test==test.max(['lev'])).max(['lev']).rename('bottom_salt')
    
    relative_depth = (mld/bottom_depth).rename('relative_depth')
    
    calc_props[i] = xr.merge([bottom_depth, bottom_sigma2, bottom_temp, bottom_salt, relative_depth, mld])

# Read in observations

In [13]:
 ## Observations 
# Read WOA using opendap
Temp_url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/WOA/WOA13/1_deg/annual/temp'
Salt_url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/WOA/WOA13/1_deg/annual/salt'
#Oxy_url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/WOA/WOA13/5_deg/annual/doxy'

ds_obs = xr.merge([
    xr.open_dataset(Temp_url).tmn.load(),
    xr.open_dataset(Salt_url).smn.load()])

cthetao_obs = xr.apply_ufunc(gsw.CT_from_pt, ds_obs.smn, ds_obs.tmn, dask='parallelized',
                                             output_dtypes=[float,]).rename('cthetao_obs').to_dataset() 
pdens_obs = xr.apply_ufunc(gsw.density.sigma2, ds_obs.smn, cthetao_obs.cthetao_obs, dask='parallelized',
                                             output_dtypes=[float,]).rename('pdens_obs').to_dataset()

ds_obs = xr.merge([ds_obs, cthetao_obs, pdens_obs])

temp_obs = ds_obs.pdens_obs + ds_obs.lev
    
bottom_dens_obs = ds_obs.pdens_obs.where(temp_obs==temp_obs.max(['lev'])).max(['lev']).isel(time=-1)
bottom_temp_obs = ds_obs.tmn.where(temp_obs==temp_obs.max(['lev'])).max(['lev']).isel(time=-1)
bottom_salt_obs = ds_obs.smn.where(temp_obs==temp_obs.max(['lev'])).max(['lev']).isel(time=-1)
topo_obs = (temp_obs- ds_obs.pdens_obs).isel(time=-1).max(['lev']).rename('topo')

bottom_props_obs = xr.merge([bottom_dens_obs, bottom_temp_obs, bottom_salt_obs, topo_obs]) 

/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/srv/conda/envs/notebook/lib/python3.7/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


In [14]:
# get grid information from observations to regrid
[lat_obs, lon_obs] = np.meshgrid(bottom_dens_obs.lat, bottom_dens_obs.lon)
ds_out = xr.Dataset({'lat': (['x','y'],lat_obs),
                     'lon': (['x','y'],lon_obs)})

In [15]:
# regrid models to observation grid
regridded_props = {}

for i in calc_props:
    ds_in = xr.Dataset({'lat': (['x','y'],thetaodict[i].lat),
                        'lon': (['x','y'],thetaodict[i].lon)})
    
     # Regrid the data
    f = xe.Regridder(ds_in, ds_out, np.str('nearest_s2d'), periodic=True, reuse_weights=True)
    bottom_sigma2_regridded = f(calc_props[i].bottom_sigma2)
    relative_depth_regridded = f(calc_props[i].relative_depth)
    mld_regridded = f(calc_props[i].mld)

    regridded_props[i] = xr.merge([bottom_sigma2_regridded, relative_depth_regridded, mld_regridded])

Reuse existing file: nearest_s2d_1080x1440_360x180_peri.nc
Reuse existing file: nearest_s2d_384x320_360x180_peri.nc
Reuse existing file: nearest_s2d_332x362_360x180_peri.nc
Reuse existing file: nearest_s2d_330x360_360x180_peri.nc
Reuse existing file: nearest_s2d_294x362_360x180_peri.nc
Reuse existing file: nearest_s2d_330x360_360x180_peri.nc
Reuse existing file: nearest_s2d_292x362_360x180_peri.nc
Reuse existing file: nearest_s2d_384x320_360x180_peri.nc
Reuse existing file: nearest_s2d_291x360_360x180_peri.nc
Reuse existing file: nearest_s2d_292x362_360x180_peri.nc
Reuse existing file: nearest_s2d_294x362_360x180_peri.nc


In [16]:
# give obs the same lat/lon names
bottom_props_obs = bottom_props_obs.rename({'lat':'y','lon':'x'})

# Figures

In [ ]:
# load regridded variables into memory for the time period we want
# this will make the figure plotting below quicker
for i in thetaodict:
       regridded_props[i].isel(time=0).load()

distributed.protocol.core - CRITICAL - Failed to Serialize
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/protocol/core.py", line 44, in dumps
    for key, value in data.items()
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/protocol/core.py", line 45, in <dictcomp>
    if type(value) is Serialize
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/protocol/serialize.py", line 167, in serialize
    for obj in x
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/protocol/serialize.py", line 167, in <listcomp>
    for obj in x
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/protocol/serialize.py", line 167, in serialize
    for obj in x
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/protocol/serialize.py", line 167, in <listcomp>
    for obj in x
  File "/srv/conda/envs/notebook/lib/python3.7/site-packages/dis

In [ ]:
fig =plt.figure(figsize=(14,14))


for n, i in enumerate(thetaodict):
    ax = plt.subplot(4,3,n+2,projection = ccrs.SouthPolarStereo())
    ax.set_extent([0.005, 360, -90, -50], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND,zorder=100,edgecolor='k')
   
    diff = regridded_props[i].mld
    diff = ma.masked_where(lat_obs>0.,diff)
    this = ax.pcolormesh(lon_obs,lat_obs,diff,transform=ccrs.PlateCarree(),vmin=50,vmax=500)

       
    plt.title(np.str(thetaodict[i].source_id))
    fig.colorbar(this,ax=ax)
plt.tight_layout()
plt.show()
fig.savefig('mld')
plt.close()

In [ ]:
fig =plt.figure(figsize=(14,14))

ax = plt.subplot(4,3,1,projection = ccrs.SouthPolarStereo())
ax.set_extent([0.005, 360, -90, -50], crs=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND,zorder=100,edgecolor='k')
     

bottom_props_obs.pdens_obs.plot(cmap = plt.cm.viridis,
                         transform=ccrs.PlateCarree(),vmin=36.8, vmax=37.2)
ax.set_title('Observations')

for n, i in enumerate(thetaodict):
    ax = plt.subplot(4,3,n+2,projection = ccrs.SouthPolarStereo())
    ax.set_extent([0.005, 360, -90, -50], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND,zorder=100,edgecolor='k')
   
    diff = bottom_props_obs.pdens_obs-regridded_props[i].bottom_sigma2
    print(diff.shape)
    diff.plot(cmap = plt.cm.RdBu_r,transform=ccrs.PlateCarree(),
                         vmin=-.3, vmax=.3)
    
    regridded_props[i].mld.values = ma.masked_where(lat_obs>0.,regridded_props[i].mld.values)
    regridded_props[i].mld.plot.contour(transform=ccrs.PlateCarree())
       
    plt.title(np.str(thetaodict[i].source_id))
plt.tight_layout()
plt.show()
fig.savefig('bottom_density_diff_polar_contour')
plt.close()

In [ ]:
fig =plt.figure(figsize=(14,14))

ax = plt.subplot(4,3,1)

bottom_props_obs.pdens_obs.isel(y=slice(0,12)).plot(cmap = plt.cm.viridis,
                         vmin=36.8, vmax=37.2)
ax.set_title('Observations')

for n, i in enumerate(thetaodict):
    ax = plt.subplot(4,3,n+2)
    diff = bottom_props_obs.pdens_obs-regridded_props[i].bottom_sigma2
    
    diff.isel(y=slice(0,12)).plot(cmap = plt.cm.RdBu_r,
                         vmin=-.3, vmax=.3)
       
    plt.title(np.str(thetaodict[i].source_id))
plt.tight_layout()
plt.show()
fig.savefig('bottom_density_diff')
plt.close()

In [ ]:
fig =plt.figure(figsize=(14,14))

ax = plt.subplot(4,3,1,projection = ccrs.SouthPolarStereo())
ax.set_extent([0.005, 360, -90, -50], crs=ccrs.PlateCarree())
    

bottom_props_obs.pdens_obs.plot(cmap = plt.cm.viridis,
                         transform=ccrs.PlateCarree(),vmin=36.8, vmax=37.2)
ax.set_title('Observations')

for n, i in enumerate(thetaodict):
    ax = plt.subplot(4,3,n+2,projection = ccrs.SouthPolarStereo())
    ax.set_extent([0.005, 360, -90, -50], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.LAND,zorder=100,edgecolor='k')
   
    diff = regridded_props[i].bottom_sigma2
    print(diff.shape)
    diff.plot(transform=ccrs.PlateCarree(),
                         vmin=36.8, vmax=37.2)
    
    regridded_props[i].mld.plot.contour(transform=ccrs.PlateCarree())
       
    plt.title(np.str(thetaodict[i].source_id))
plt.tight_layout()
plt.show()
fig.savefig('bottom_density_abs_polar')
plt.close()